In [157]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.applications.vgg16 import decode_predictions
from threading import current_thread, Thread, Lock
from multiprocessing import Queue

In [158]:
categories=['train','test']
categories1=['without_mask','with_mask']


In [159]:
data1=[]
for category in categories1:
    path=os.path.join('Dataset',category)
    label1=categories1.index(category)
    for file in os.listdir(path):
       img_path=os.path.join(path,file)
       img=cv2.imread(img_path)
       img=cv2.resize(img,(224,224))
       data1.append([img,label1])
    

In [160]:
data=[]
cate=[]
for category in categories:
    path=os.path.join('Dataset',category)
    for file in os.listdir(path):
        if not file.startswith('.'):
            img_path=os.path.join(path,file)
            cate.append(file)
            label=cate.index(file)
            files=os.listdir(img_path)
            for i in files:
                p=os.path.join(img_path,i)
                img_path=os.path.join(path,file)
                #print(p)
                img=cv2.imread(p)
                img=cv2.resize(img,(224,224))
                data.append([img,label])


In [161]:
import random

In [162]:
random.shuffle(data1)

In [163]:
x1=[]
y1=[]
for features1,label1 in data1:
    x1.append(features1)
    y1.append(label1) 

In [164]:
import numpy as np

In [165]:
x1=np.array(x1)
y1=np.array(y1)

In [166]:
x1.shape
y1.shape

(1965,)

In [167]:
x1=x1/255

In [168]:
x1[0]

array([[[0.40392157, 0.38823529, 0.27843137],
        [0.40784314, 0.39215686, 0.28235294],
        [0.41568627, 0.4       , 0.28627451],
        ...,
        [0.94117647, 0.97254902, 0.96862745],
        [0.96862745, 0.99215686, 0.98823529],
        [0.96862745, 0.99215686, 0.98823529]],

       [[0.40392157, 0.38823529, 0.27843137],
        [0.40784314, 0.39215686, 0.28235294],
        [0.41568627, 0.4       , 0.28627451],
        ...,
        [0.94509804, 0.97254902, 0.97254902],
        [0.96862745, 0.99215686, 0.98823529],
        [0.96862745, 0.99215686, 0.98823529]],

       [[0.40392157, 0.38823529, 0.27843137],
        [0.40784314, 0.39215686, 0.28235294],
        [0.41568627, 0.4       , 0.28627451],
        ...,
        [0.94509804, 0.97647059, 0.97647059],
        [0.96862745, 0.99215686, 0.98823529],
        [0.96862745, 0.99215686, 0.98823529]],

       ...,

       [[0.27843137, 0.26666667, 0.18431373],
        [0.28235294, 0.27058824, 0.18039216],
        [0.29019608, 0

In [169]:
from sklearn.model_selection  import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.2)

In [170]:
x_train1.shape
x_test1.shape

(393, 224, 224, 3)

In [171]:
batch_size = 50
num_train_images = x_train1.shape[0]/2
num_test_images = x_test1.shape[0]/2
num_train_threads = int(num_train_images/batch_size)  # 50
num_test_threads = int(num_test_images/batch_size)    # 25
lock = Lock()

In [172]:
def initialize_queue():
    queue = Queue()
    return queue

In [173]:
def get_img_array_labels(fpaths, queue):
    img_array=None
    for i in range(0,x_train1.shape[0]):
        if img_array==None:
            img_array = x_train1[i]
        else:
            img_array = np.vstack((img_array, x_train1[i]))
    queue.put((img_array, y_train1))
    

In [174]:
def get_img_array_labels1(fpaths, queue):
    img_array=None
    for i in range(0,x_test1.shape[0]):
        if img_array==None:
            img_array = x_test1[i]
        else:
            img_array = np.vstack((img_array, x_train1[i]))
    queue.put((img_array, y_test1))

In [175]:
def get_img_array(fpaths, queue):
    img_array = None
    for i in range(0,x_train1.shape[0]):
        if img_array is None:
            img_array = img
        else:
            img_array = np.vstack((img_array, img))        
    queue.put(img_array)

In [176]:
def get_img_array1(fpaths, queue):
    img_array = None
    for i in range(0,x_test1.shape[0]):
        if img_array is None:
            img_array = img
        else:
            img_array = np.vstack((img_array, img))        
    queue.put(img_array)

In [177]:
def dump_array(fname,arr):
    with open(fname,'wb') as f:
        pickle.dump(arr,f)

In [178]:
def get_training_data():
    threads_list = list()
    train_x = None
    train_y = []
    queue = initialize_queue()
    # iterate over num of threads to create
    for thread_index in range(num_train_threads):
        start_index = thread_index * batch_size
        end_index = (thread_index + 1) * batch_size
        file_batch = x_train1[start_index:end_index]
        thread = Thread(target =get_img_array_labels, args=(file_batch, queue))
        thread.start()
        print("Thread: {}, start index: {}, end index: {}".format(thread.name, start_index, end_index))
        threads_list.append(thread)
    
    # join threads
    for t in threads_list:
        t.join()
    while not queue.empty():
        arr, labels = queue.get()
        train_y.extend(label1)
        if train_x is None:
            train_x = arr
        else:
            train_x = np.vstack((train_x, arr))
    return train_x, train_y

In [179]:
def get_testing_data():
    threads_list = list()
    test_x = None
    queue = initialize_queue()
    # iterate over num of threads to create
    for thread_index in range(num_test_threads):
        start_index = thread_index * batch_size
        end_index = (thread_index + 1) * batch_size
        file_batch = x_test1[start_index:end_index]
        thread = Thread(target =get_img_array1, args=(file_batch, queue))
        thread.start()
        print("Thread: {}, start index: {}, end index: {}".format(thread.name, start_index, end_index))
        threads_list.append(thread)
        for t in threads_list:
            t.join()
        print("Thread: {} joined", t.name)
    while not queue.empty():
        arr= queue.get()
        if test_x is None:
            test_x = arr
        else:
            test_x = np.vstack((test_x, arr))
    return test_x

In [180]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
vgg1=VGG16()

In [ ]:
vgg1.summary()

In [ ]:
from keras import Sequential

In [ ]:
model1=Sequential()

In [ ]:
for layer in vgg1.layers[:-1]:
    model1.add(layer)

In [ ]:
model1.summary()

In [ ]:
for layer in model1.layers:
    layer.trainable=False

In [ ]:
model1.summary()

In [ ]:
from keras.layers import Dense

In [ ]:
model1.add(Dense(1,activation='sigmoid'))

In [ ]:
model1.summary()

In [ ]:
model1.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
x_train1,y_train1=get_training_data()

In [ ]:
x_test1,y_test1=get_testing_data()

In [ ]:
model1.fit(x_train1,y_train1,epochs=3,validation_data=(x_test1,y_test1))